# Connect to Hive, Presto & Trino Engine using `Python3`


#### Topics covered in this example
* Installing `python3-devel` and `cyrus-sasl-devel` on the EMR master node
* Installing python libraries on the Amazon EMR cluster
* Connecting to Hive using Python3 `PyHive` library
* Connecting to Presto using Python3 `PyHive` library
* Connecting to Trino using Python3 `PyHive` library

## Table of Contents:

1. [Prerequisites](#Prerequisites)
2. [Introduction](#Introduction)
3. [Install dependency libraries](#Install-dependency-libraries)
4. [Connect to Hive using `PyHive`](#Connect-to-Hive-using-PyHive)
5. [Connect to Presto using `PyHive`](#Connect-to-Presto-using-PyHive)
6. [Connect to Trino using `PyHive`](#Connect-to-Trino-using-PyHive)

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* This example installs python3 libraries, hence the EMR cluster attached to this notebook must have internet connectivity.
* The EMR cluster attached to this notebook should have the following packages installed:
  * `python3-devel`
  * `cyrus-sasl-devel`
  
* You can do this by running the following command on EMR mater node:
  <blockquote><p style="font-family:'Courier New'">sudo yum install -y python3-devel cyrus-sasl-devel</p></blockquote>
* This notebook uses the `Python3` kernel.
***

## Introduction
In this example we use `Python3` to connect to a table in Hive, Presto and Trino using `PyHive`.

PyHive is a collection of Python DB-API and SQLAlchemy interfaces for Presto and Hive.
***

## Install dependency libraries

`%pip install` is the same as `!/emr/notebook-env/bin/pip install` and are installed in `/home/emr-notebook/`.

After installation, these libraries are available to any user running an EMR notebook attached to the cluster. Python libraries installed this way are available only to processes running on the master node. The libraries are not installed on core or task nodes and are not available to executors running on those nodes.

In [ ]:
%pip install pyhive thrift sasl thrift_sasl

## Connect to Hive using `PyHive`

We will connect to Hive using `PyHive` library. Please make sure you replace the values for `hostName, userName, and databaseName` as applicable to your environment. The Hiveserver2 port is set to default 10000.

In this example, we are connecting to Hive database `default` running locally on the EMR master node. We will query the table `hive_sample_table` to retrieve some values.

In [ ]:
from pyhive import hive
import getpass

hostName = "127.0.0.1"
userName = "hadoop"
userPassword = getpass.getpass('Enter hive user password')
databaseName = "default"
hivePort = 10000

def hiveConnection(hostName, hivePort, userName, userPassword, databaseName):
    conn = hive.connect(host=hostName,
                           port=hivePort,
                           username=userName,
                           password=userPassword,
                           database=databaseName,
                           auth='CUSTOM')
    cur = conn.cursor()
    cur.execute('SELECT id FROM hive_sample_table LIMIT 3')
    result = cur.fetchall()

    return result

# Call above function
tableData = hiveConnection(hostName, hivePort, userName, userPassword, databaseName)

# Print the results
print(tableData)

## Connect to Presto using `PyHive`

We will connect to Presto using `PyHive` library. Please make sure you replace the values for `hostName, userName, schemaName and catalogName` as applicable to your environment. The port is set to EMR default of 8889.

In this example, we are connecting to `default` schema/database stored inside the `hive` catalog on the EMR master node. We will query the table `presto_sample_table` using within the `default` schema to retrieve some values.

The connection uses HTTP protocol for Presto. You can enabled SSL/TLS and configure LDAPS for Presto on Amazon EMR by referring to the documentation [here](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/presto-ssl.html)

Presto authentication and without authentication and getPass #####

In [ ]:
from pyhive import presto
import requests

hostName = "127.0.0.1"
userName = "hadoop"
schemaName = "default"
catalogName = "hive"
prestoPort = 8889

headers = {
    'X-Presto-User': userName,
    'X-Presto-Schema': schemaName,
    'X-Presto-Catalog': catalogName
}

prestoSession = requests.Session()
prestoSession.headers.update(headers)

def prestoConnection(prestoSession, hostName, prestoPort):
    conn = presto.connect(requests_session=prestoSession,
                          host=hostName,
                          port=prestoPort
                          )

    cur = conn.cursor()
    cur.execute('SELECT id FROM presto_sample_table LIMIT 3')
    result = cur.fetchall()

    return result

# Call above function
tableData = prestoConnection(prestoSession, hostName, prestoPort)

# Print the results
print(tableData)

## Connect to Trino using `PyHive`

We will now connect to Trino using `PyHive` library. Please make sure you replace the values for `hostName, userName, schemaName and catalogName` as applicable to your environment. The port is set to EMR default of 8889.

In this example, we are connecting to `default` schema/database stored inside the `hive` catalog on the EMR master node. We will query the table `trino_sample_table` using within the `default` schema to retrieve some values.

The connection uses HTTP protocol for Presto. You can enabled SSL/TLS and configure LDAPS for Presto on Amazon EMR by referring to the documentation [here](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/presto-ssl.html)

In [ ]:
from pyhive import trino
import requests

hostName = "127.0.0.1"
userName = "hadoop"
schemaName = "default"
catalogName = "hive"
trinoPort = 8889

## Starting with Trino release 0.351 rename client protocol headers to start with X-Trino- instead of X-Presto-
headers = {
    'X-Presto-User': userName,
    'X-Presto-Schema': schemaName,
    'X-Presto-Catalog': catalogName
}

trinoSession = requests.Session()
trinoSession.headers.update(headers)

def trinoConnection(trinoSession, hostName, trinoPort):
    conn = trino.connect(requests_session=trinoSession,
                         host=hostName,
                         port=trinoPort
                         )

    cur = conn.cursor()
    cur.execute('SELECT id FROM trino_sample_table LIMIT 3')
    result = cur.fetchall()

    return result

# Call above function
tableData = trinoConnection(trinoSession, hostName, trinoPort)

# Print the results
print(tableData)